In [1]:
import os  
import pandas as pd
import glob  
from datetime import datetime
from dateutil.parser import parse
from sklearn.feature_extraction.text import HashingVectorizer
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

In [2]:
def to_month_day(date_str):
    try:
        # 날짜를 파싱하여 datetime 객체로 변환
        # print(date_str)
        parsed_date = parse(date_str)
        # "월-일" 형식으로 변환
        month_day = parsed_date.strftime("%m-%d")
        return month_day
    except ValueError:
        print("exception occurs")
        return None  # 잘못된 날짜 형식에 대한 처리

def to_hour_minute(time_str):
    try:
        # 시간을 파싱하여 datetime 객체로 변환
        parsed_time = datetime.strptime(time_str, '%H:%M:%S' if ':' in time_str[5:] else '%H:%M')
        # "시간:분" 형식으로 변환
        hour_minute = parsed_time.strftime("%H:%M")
        return hour_minute
    except ValueError:
        return None  # 잘못된 시간 형식에 대한 처리

def to_address(x):
    y = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", x)
    return y

def to_hour(x):
    return int(x[:2])

def to_dateM(x):
    y = int(x[:2]) * 30 + int(x[3:])
    return y

In [17]:
def load_process(combined_dataframe):

    # preprocess
    combined_dataframe['기온'] = combined_dataframe['기온'].fillna(combined_dataframe['기온(℃)'])
    combined_dataframe['기온'] = combined_dataframe['기온'].fillna(combined_dataframe['기온 (℃)'])
    combined_dataframe['습도'] = combined_dataframe['습도'].fillna(combined_dataframe['습도(%)'])
    combined_dataframe['습도'] = combined_dataframe['습도'].fillna(combined_dataframe['습도 (%)'])
    combined_dataframe['재비산먼지 평균농도'] = combined_dataframe['재비산먼지 평균농도'].fillna(combined_dataframe['재비산먼지평균농도(㎍/㎥)'])
    combined_dataframe['재비산먼지 평균농도'] = combined_dataframe['재비산먼지 평균농도'].fillna(combined_dataframe['재비산먼지 평균농도(㎍/㎥)'])
    combined_dataframe['재비산먼지 평균농도'] = combined_dataframe['재비산먼지 평균농도'].fillna(combined_dataframe['재비산먼지평균농도'])
    combined_dataframe = combined_dataframe.drop(labels=['H', '재비산먼지평균농도', '기온(℃)', '습도(%)', '재비산먼지평균농도(㎍/㎥)', '기온 (℃)', '습도 (%)', '재비산먼지 평균농도(㎍/㎥)'], axis=1)

    df = combined_dataframe.dropna()

    # feature engineering
    df['date'] = df['측정일자'].apply(to_month_day)
    df['time'] = df['측정시간'].apply(to_hour_minute)
    df = df.drop(['측정일자', '측정시간'], axis=1)
    df['start_city'] = df['시작점'].apply(to_address)
    df['end_city'] = df['종점'].apply(to_address)
    df = df.drop(['지역', '지역명', '도로명', '시작점', '종점'], axis=1)

    hash_vectorizer = HashingVectorizer(n_features=3)  # n_features는 원하는 차원 수
    address_hashed1 = hash_vectorizer.fit_transform(df['start_city'])
    address_hashed2 = hash_vectorizer.fit_transform(df['end_city'])

    df[['start_city_c1', 'start_city_c2', 'start_city_c3']] = address_hashed1.toarray()
    df[['end_city_c1', 'end_city_c2', 'end_city_c3']] = address_hashed2.toarray()

    df_onehot = pd.get_dummies(df, columns=['오염범례'], drop_first=True)
    df_onehot['hour'] = df_onehot['time'].apply(to_hour)
    df_onehot['date'] = df_onehot['date'].apply(to_dateM)
    df = df_onehot.drop(['time', 'start_city', 'end_city'], axis=1)

    return df




In [24]:
def preprocess_X(df):
    # df['기온'] = df['기온'].fillna(df['기온(℃)'])
    # df['기온'] = df['기온'].fillna(df['기온 (℃)'])
    # df['습도'] = df['습도'].fillna(df['습도(%)'])
    # df['습도'] = df['습도'].fillna(df['습도 (%)'])
    # df = df.drop(labels=['H', '기온(℃)', '습도(%)', '기온 (℃)', '습도 (%)'], axis=1)

    df = df.dropna()

    # feature engineering
    df['date'] = df['측정일자'].apply(to_month_day)
    df['time'] = df['측정시간'].apply(to_hour_minute)
    df = df.drop(['측정일자', '측정시간'], axis=1)
    df['start_city'] = df['시작점'].apply(to_address)
    df['end_city'] = df['종점'].apply(to_address)
    df = df.drop(['지역', '지역명', '도로명', '시작점', '종점'], axis=1)

    hash_vectorizer = HashingVectorizer(n_features=3)  # n_features는 원하는 차원 수
    address_hashed1 = hash_vectorizer.fit_transform(df['start_city'])
    address_hashed2 = hash_vectorizer.fit_transform(df['end_city'])

    df[['start_city_c1', 'start_city_c2', 'start_city_c3']] = address_hashed1.toarray()
    df[['end_city_c1', 'end_city_c2', 'end_city_c3']] = address_hashed2.toarray()

    df_onehot = pd.get_dummies(df, columns=['오염범례'], drop_first=True)
    df_onehot['hour'] = df_onehot['time'].apply(to_hour)
    df_onehot['date'] = df_onehot['date'].apply(to_dateM)
    df = df_onehot.drop(['time', 'start_city', 'end_city'], axis=1)

    return df

In [19]:
def modeling(df):
    X = df.drop(['재비산먼지 평균농도'], axis=1)
    y = df[['재비산먼지 평균농도']]

    # 데이터셋 분할 (훈련 세트와 테스트 세트)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Random Forest 분류 모델 생성
    rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)

    # 모델 훈련
    rf.fit(X_train, y_train)

    # 예측
    y_pred = rf.predict(X_test)
    y_true = y_test['재비산먼지 평균농도']

    mae = mean_absolute_error(y_test, y_pred)
    print("\nMAE (Mean Absolute Error):", mae)

    mse = mean_squared_error(y_test, y_pred)
    print("MSE (Mean Squared Error):", mse)

    rmse = sqrt(mse)
    print("RMSE (Root Mean Squared Error):", rmse)

    return rf, y_pred, y_true

In [6]:
# all csv files
csv_files = glob.glob(os.path.join("./", "*.csv"))
dataframes = [pd.read_csv(file, encoding='cp949') for file in csv_files]

combined_dataframe = pd.concat(dataframes, ignore_index=True)




TypeError: expected str, bytes or os.PathLike object, not DataFrame

In [20]:
df = load_process(combined_dataframe)
model, y_pred, y_true = modeling(df)


C:\Users\dbgpw\AppData\Local\Temp\ipykernel_7792\3471987278.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['측정일자'].apply(to_month_day)
C:\Users\dbgpw\AppData\Local\Temp\ipykernel_7792\3471987278.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['측정시간'].apply(to_hour_minute)
C:\Users\dbgpw\AppData\Local\Temp\ipykernel_7792\1905079426.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), f


MAE (Mean Absolute Error): 24.88370819024264
MSE (Mean Squared Error): 9873.335706710774
RMSE (Root Mean Squared Error): 99.36466025056782


In [37]:
df = pd.DataFrame([['05-11', '06:17', '경기도 수원시', '경기도', '경기도 수원시 영통구 영통동', '경희대학교 국제캠퍼스', '수원역', '보통', 12, 81],
                    ['05-11', '06:17', '경기도 수원시', '경기도', '경기도 수원시 영통구 영통동', '경희대학교 국제캠퍼스', '수원역', '좋음', 12, 81],
                    ['05-11', '06:17', '강원도 양양', '강원도', '강원도 양양군', '양양', '강원도 양양군 서면', '매우좋음', 17, 40],
                    ['05-11', '06:32', '경상북도 안동', '경상북도', '경상북도 안동시', '경상북도 안동시', '경상북도 안동시 풍천면', '매우나쁨' ,13, 85],
                    ['05-11', '06:33', '서울 마곡동', '서울시', '서울시 마곡동', '서울시 마곡동', '서울 양천구 오목로', '나쁨', 17, 74],
                    ['05-11', '06:33', '부산 해운대', '부산광역시', '부산광역시 중구', '부산광역시 중구', '부산 해운대구 동백로', '보통', 15, 80],
                    ['05-11', '06:33', '경기도 성남시', '경기도', '경기도 성남시', '경기도 성남시', '경기도 수원시 팔달구', '보통', 16, 71]],
                columns=['측정일자', '측정시간', '지역', '지역명', '도로명', '시작점', '종점', '오염범례', '기온', '습도'])

test_data = preprocess_X(df)
final_pred = model.predict(test_data)
# df

In [38]:
final_pred

array([121.73401203,  69.85289268,  21.76703301, 414.90678292,
       174.27464328, 121.90769975, 121.6386999 ])